In [117]:
import torchvision
import torchvision.transforms as T

from tqdm.notebook import tqdm

from torch.utils.data import DataLoader, Dataset, Subset
import torch
import matplotlib.pyplot as plt

import numpy as np
import PIL.Image as Image

import pandas as pd

from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
import torch.nn as nn

In [118]:
split = 0.8

n = 12600

classes = 15

batch_size = 512

In [119]:
def move_to_classes():
    import shutil
    import os

    label_keys = {
    'cycling': 0, 'dancing': 1, 'drinking': 2, 'eating': 3,
    'fighting': 4, 'hugging': 5, 'laughing': 6, 'listening_to_music': 7,
    'running': 8, 'sitting': 9, 'sleeping': 10, 'texting': 11, 'using_laptop': 12,
    'calling': 13, 'clapping': 14
}
    #for d in label_keys:
        #os.mkdir(f'data/train_labeled/{d}')

    label_dataframe = pd.read_csv('data/Training_set.csv')
    label = label_dataframe.pop('label')
    
    for i in range(1, 12601 + 1):
        source_path = f'data/train/Image_{i}.jpg'

        new_folder = label[i - 1]
        new_path = f'data/train_labeled/{new_folder}/{i}.jpg'

        shutil.move(source_path, new_path)

    return None

In [120]:
try:
    move_to_classes()
except:
    print('has already moved!')

has already moved!


In [121]:
def load_dataset(batch_size, n, split):
    data_path = 'data/train_labeled'
    new_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=T.Compose([T.ToTensor(), T.Resize(size=(80, 50)), T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    )
    
    train_dataset, test_dataset = random_split(new_dataset, [int(n * split) + 1, int(n * (1 - split))])
    #train_dataset = torch.utils.data.Subset(new_dataset, range(int(b * 0.75)))
    #test_dataset = torch.utils.data.Subset(new_dataset, range(int(b * 0.75), b))
    
    #train_dataset, test_dataset = random_split(train_dataset, [30000, 20000])
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    return train_loader, test_loader

In [122]:
train_loader, test_loader = load_dataset(batch_size, n, split)

In [123]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3)
        
        self.lin1 = nn.Linear(6336, 1000)
        self.drp1 = nn.Dropout(0.5)
        self.lin2 = nn.Linear(1000, 100)
        self.lin3 = nn.Linear(100, classes)

    def forward(self, x):       
        x = F.max_pool2d(self.conv1(x), (2, 2))
        x = F.max_pool2d(self.conv2(x), 2)
        #print(x.shape)
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.lin1(x))
        x = F.relu(self.drp1(x))
        x = F.relu(self.lin2(x))
    
        x = F.softmax(self.lin3(x), dim=1)
        
        return x

def check(loader, n, split, isTrain, model):
    acc = torch.zeros(11)
    eta = 1 - split
    k = 0
    if isTrain:
        eta = split
    
    with torch.no_grad():
        for x, y in tqdm(loader):
            r = y.shape[0]
            
            pred = model(x)
            #acc = np.zeros(11)
            
            for i in range(r):
                #print(f'pred.argmax() = {pred[i].argmax()}, y = {y[i]}')
                if pred[i].argmax() == y[i]:
                    k += 1
    #print(k)
    
    return k * 100 / n / eta

In [124]:
alpha = 3e-4

model = Net()
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=alpha)

history = []
acc = []
epochs = 50

In [127]:
#epochs = 4000
#for g in optim.param_groups:
#    g['lr'] = 1e-5
acc_train = []
acc_test = []

history = []
for epoch in range(epochs):
    for x_batch, y_batch in tqdm(train_loader):
        y_pred = model(x_batch)
    
        loss = loss_fn(y_pred, y_batch)
        #train_acc = torch.sum(y_pred == y_batch)
        optim.zero_grad()
        loss.backward()
        optim.step()

    
    acc_test.append(check(test_loader, n, split, False, model))
    print(f'acc on test[{epoch}]: {acc_test[epoch]}%')

    #acc_train.append(check(train_loader, a, b, split, True, model))
    #print(f'acc on train[{epoch}]: {acc_train[epoch]}%')
        
    history.append(loss.item())
        
    print(f'Epoch: {epoch + 1}/{epochs}; loss: {loss.item()};') 

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

acc on test[0]: 28.730158730158738%
Epoch: 1/50; loss: 2.2280163764953613;


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

acc on test[30]: 28.809523809523817%
Epoch: 31/50; loss: 2.1667487621307373;


  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [130]:
check(train_loader, n, split, True, model)

  0%|          | 0/20 [00:00<?, ?it/s]

66.4484126984127